## Java Threads

We are going to use Java for lectures on concurrency control and mutual exclusion. This is a very mechanistic description of how to make and call threads in Java.


### iJava

There is no conda install for ijava. So we must do it manually. It is in python so pretty portable.  The linux-y version looks something like.
```
conda install -c conda-forge openjdk
cd /tmp
wget https://github.com/SpencerPark/IJava/releases/download/v1.3.0/ijava-1.3.0.zip
python install.py --sys-prefix
```

### Java threads, the mechanics

* To run a Java thread, you define a `Thread` object and call `start()` on the object.
    * the `Thread` class takes a `Runnable` object in it's constructor
* `Runnable` is an interface that any class can implement
  * must include a `run` method as an entry point.
* `thread.start()` is the fork/exec analogy
  * it returns immediately in the calling thread
  * and invokes `Runnable.run()` in a new thread
  
The following is a minimal example for creating parallel threads in Java.  

_Note_: the snippet uses `StringIO` instead of `System.out.println` because I/O is weird in IJava notebooks.

In [1]:
import java.io.StringWriter;

class SimpleConc implements Runnable
{
  int thread_id;    // Variable containing specific id of this thread.
  StringWriter outstream;
  
  // Run: overides Runnabale.Run, thread entry point
  public void run ()
  {
    this.outstream.write( thread_id + ": Running thread\n" );
  }

  // Constructor: set thread id
  SimpleConc ( int id, StringWriter outstream )
  {
    this.thread_id = id;
    this.outstream = outstream;
  }
}

int numthreads = 8;

StringWriter outstream = new StringWriter();

// create and start specified thread objects of class SimpleConc
for ( int i=1; i<=numthreads; i++ )
{
    new Thread ( new SimpleConc(i, outstream) ).start();
}

System.out.println(outstream)

1: Running thread
4: Running thread
5: Running thread
2: Running thread
3: Running thread
6: Running thread
7: Running thread
8: Running thread



**Small Caution** Java has multiple ways to create threads.  The other way (not shown) is to inherit from or extend the `Thread` class.  This is inferior. Because `Thread` is a class, you cannot extend `Thread` and another class, e.g. `Applet`.  Good practice is to only inherit when you want to extend base class functionality.  That's not what we're doing.

Use Runnable: it’s preferable.

### Awaiting Thread Completion

* `join()` a thread
  * wait for a thread to finish.
  * return immediately if already finished.
  * finished threads __are not__ garbage collected (until they are dereferenced)
    * you can call functions on `join()`ed threads


In [2]:
class SummingShared implements Runnable
{
  // Create shared variable
  static int sharedvar = 0;

  public void run ()
  { 
    for ( int i=0; i<1000000; i++ )
    {
      sharedvar++;
    }
  }
}

int numthreads = 4;

// Array to hold references to thread objects
Thread[] threads = new Thread[numthreads];

// create and start specified thread objects of class SynchronizedWorks
for ( int i=0; i<numthreads; i++ )
{
    threads[i] = new Thread ( new SummingShared() );
    threads[i].start();
}

// Await the completion of all threads
for ( int i=0; i<numthreads; i++ )
{
    try
    {
        threads[i].join();
    }
    catch (InterruptedException e)
    {
        System.out.println("Thread interrupted.  Exception: " + e.toString() +
                           " Message: " + e.getMessage()) ;
        return -1;
    }
}

System.out.println("All threads complete.");
System.out.println("Shared variable = " + SummingShared.sharedvar);

System.out.println("Thread 0 is alive? " + threads[0].isAlive())

All threads complete.
Shared variable = 1601933
Thread 0 is alive? false


This example get references to the thread objects which it can manipulate.  Joins them and then reports the aggregate sum when all threads are complete.

We see that the threads exist after they are joined.

But, we also see that we got the "wrong" answer.  That brings us to synchronization.